In [ ]:
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)

# Training an Agent through Imitation Learning

By the end of this tutorial you will know what Imitation learning algorithms are provided in this library and you will know how to use Deep Inverse Reinforcement Learning in conbination with any RL agent of the library.

This library provides three Imitation Learning algorithms:

## 1) Deep Inverse Reinforcement Learning (DeepIRL): 


It Consist of an implementation of "Apprenticeship Learning" algorithm from Pieter Abbeel and Andrew Y. Ng. 2004. Apprenticeship learning via inverse reinforcement learning. ICML '04."

As an overview, this algorithm have two main entities with two adversarial task: 1) a RL agent generate actions that aims to be very similar to expert actions. 2) a discriminator tries to diferenciate what actions comes from a RL agent and what actions comes from an expert. This task produces as result a value that is used as reward to train the RL agent.

This particular implememtation uses Deep Learning. For this purpose whe have replaced the classificator used for the discriminator in the original work by a neural network. This algorithm is compatible with all Deep Reinforcement Learning agents in this library.

## 2) Generative Adversarial Imitation Learning (GAIL)

This is an implementation of "HO, Jonathan; ERMON, Stefano. Generative adversarial imitation learning. Advances in neural formation processing systems, 2016, vol. 29, p. 4565-4573." 

This algorithm is very similar to DeepIRL but use the workflow of Trus Region Policy Optimization (TRPO) algorithm (this is another RL algorithm not included in the library) to makes the process more efficient. Have two main entities: 1) a reinforcement learning agent that generates actions that aims to be very similar to the expert actions. 2) a discriminator neural network that tries to diferenciate what actions comes from a RL agent and what comes from an expert. This task produces as result a value that is used as reward to train the RL agent. 

This particular implementation uses Proximal Policy Optimizarion (PPO) instead of TRPO because PPO was created as a refined version of TRPO and both have the same workflow. (This means that GAIL is only compatible with PPO and no other RL agent can be used with it)

## 3) Behavioral Cloning

This algorithm consist of a supervised deep learning problem where a neural network is trained using a dataset of expert experiences which contains the states paired with actions. The neural network is trained using the states as inputs and the actions as labels.

In this library we provide the tools to train the RL agents through behavioral cloining. This tolls also allows to pretrain a RL agent over labeled data and then make fine tuning with RL or IRL.

## Expert Data

All Imitation learning methods need a dataset of expert demonstrations. This dataset should contain the experiences on each time step. This experiences depending on the problem, may contain only the states of the states paired with actions. We also provides some utilities to store and load the exper datasets.

## Soving a Problem through DeepIRL

In this notebook we introduce the use of Inverse Reinforcement Learning algorithms, specifically DeepIRL. ILR algorithms are used to infere a reward function for training a RL agent so these method are composed by a neural network that infere the reward, an environment and a RL agent. We weel see how to join all this stuff the next sections:

- Expert Experiences Dataset
    - Collecting the Exper Experiences
    - Loading the Expert Experiences
- Define the RL Problem
    - Define the environment
    - Defining the Agent's Neural Network Architecture
    - Define the RL Agent
    - Build a RL Problem
- Define the IRL Problem
    - Defining Discriminator Neural Network
    - Build the IRL Problem
    - Solving the IRL Problem

In [ ]:
from RL_Problem import rl_problem
from IL_Problem.gail import GAIL
from IL_Problem.deepirl import DeepIRL
from RL_Agent import ppo_agent_discrete_parallel, dpg_agent
from IL_Problem.base.utils.callbacks import load_expert_memories, Callbacks
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, LSTM, Input
from RL_Agent.base.utils import agent_saver
from RL_Agent.base.utils.networks import networks as rl_networks
from IL_Problem.base.utils.networks import networks_dictionaries as il_networks
import gym

# Expert Experiences Dataset

## Collecting the Expert Experiences (only if needed)

We provide an expert demosntartions dataset in "tutorials/tf_tutorials/expert_demonstrations/ExpertLunarLander.pkl". This dataset was created runing an already trained DPG agent over the environment.

Next, we provide the code we have used to generate the dataset with a DPG agent. If you already have a dataset, you do not need to run the next cell. In this code we instantiate a RL problem to train an agent and pass some callbacks to record the experiences in test function. We provide this callbacks in "IL_Problem.base.utils.callbacks.py"

In [ ]:
environment = "LunarLander-v2"
environment = gym.make(environment)

exp_path = "tutorials/tf_tutorials/expert_demonstrations/ExpertLunarLander.pkl"
net_architecture = rl_networks.net_architecture(dense_layers=2,
                                           n_neurons=[256, 256],
                                           dense_activation=['relu', 'relu'])

expert = dpg_agent.Agent(learning_rate=5e-4,
                         batch_size=32,
                         net_architecture=net_architecture)

expert_problem = rl_problem.Problem(environment, expert)

callback = Callbacks()

# Comentar si ya se dispone de un fichero de experiencias como "Expert_LunarLander.pkl"
print("Comienzo entrenamiento de un experto")
expert_problem.solve(1000, render=False, max_step_epi=250, render_after=980, skip_states=3)
expert_problem.test(render=False, n_iter=400, callback=callback.remember_callback)

callback.save_memories(exp_path)

# Define a RL Problem

## Define the environment
We are going to use the LunarLander environment from OpenAI Gym. 

In [ ]:
environment = "LunarLander-v2"
environment = gym.make(environment)

## Defining the Agent's Neural Network Architecture

We defined only one network architecture because both actor and critic networks will have the same architecture.

In [ ]:
def lstm_custom_model(input_shape):
    actor_model = Sequential()
    actor_model.add(LSTM(16, input_shape=input_shape, activation='tanh'))
    actor_model.add(Dense(256, input_shape=input_shape, activation='relu'))
    actor_model.add(Dense(256, activation='relu'))
    return actor_model


In [ ]:
net_architecture = rl_networks.ppo_net(use_custom_network=True,
                                        actor_custom_network=lstm_custom_model,
                                        critic_custom_network=lstm_custom_model
                                        )

## Define the RL Agent

Here, we define the RL agent. A using the next parameters:

* actor_lr: learning rate for training the actor neural network.
* critic_lr: learning rate for training the neural network.
* batch_size: Size of the batches used for training the neural network. 
* memory_size: Size of the buffer filled with experiences in each algorithm iteration. 
* epsilon: Determines the amount of exploration (float between [0, 1]). 0 -> Full Exploitation; 1 -> Full exploration.
* epsilon_decay: Decay factor of the epsilon. In each iteration we calculate the new epslon value as: epsilon' = epsilon * epsilon_decay.
* esilon_min: minimun value epsilon can reach during the training procedure.
* net_architecture: net architecture defined before.
* n_stack: number of stacked timesteps to form the state.

In [ ]:
agent = ppo_agent_discrete_parallel.Agent(actor_lr=1e-4,
                                  critic_lr=1e-4,
                                  batch_size=128,
                                  memory_size=1024,
                                  epsilon=0.9,
                                  epsilon_decay=0.97,
                                  epsilon_min=0.15,
                                  net_architecture=net_architecture,
                                  n_stack=discriminator_stack)

## Build a RL Problem

Create a RL problem were the comunications between agent and environment are managed. In this case, we use the funcionality from "RL_Problem.rl_problem.py" which makes transparent to the user the selection of the matching problem. The function "Problem" automaticaly selects the problem based on the agent used.

In [ ]:
rl_problem = rl_problem.Problem(environment, agent)

# Define the IRL Problem

Lets see how to create a IRL problem. Here we need to create a neural network for the discriminator and solve the RL problem with the reward calculated by the discriminator.

## Defining Discriminator Neural Network

The procedures for defining the neural network for the discriminator are the same that those that we have seen in all past tutorials for the RL agent network. The main difference is that utilities are found inside the "IL_Problem.base" folder.

As we did for the RL agent, we can define the neural network architecture creating a keras model:

In [ ]:
def one_layer_custom_model(input_shape):
    x_input = Input(shape=input_shape, name='disc_s_input')
    x = Dense(128, activation='relu')(x_input)
    x = Dense(128, input_shape=input_shape, activation='relu')(x)
    x = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=x_input, outputs=x)
    return model

The functionality for creating the dictionary for the network architecture can be found in "IL_Problem.base.utils.networks.networks_dictionaries.py". The parameters to set the keys of the dictionary are equivalent to those for the RL agent. 

As happended with some RL agent, the discriminator neural network is splited in three subnetworks to allow the use of state or state + actions as inputs. The three subnetworks are:

- state_network: It recieves the state as input (1D, 2D, 3D array). Is not mandatory to implement it but is the only one wich allows 2D or 3D inputs. 
- action_network: It recieves the actions as input (1D array). This subnetwork only works if the algorithm is using pairs state-actions to infere the reward, when are used only state this subnetwork does not works. 
- common_network: It recieves the preprocessed data from state and action networks. This subnetwork is mandatory and recieves a 1D array formed by the output of state network or by the contatenation of the outputs of the state and action networks.

In [ ]:
irl_net_architecture = il_networks.irl_discriminator_net(use_custom_network=True,
                                                         common_custom_network=one_layer_custom_model,
                                                         define_custom_output_layer=True,
                                                         use_tf_custom_model=False)

## Loading the Expert Experiences

In "IL_Problem.base.utils.callbacks.py" we have some utilities for storing and loading expert experiences. Especifically, we use the function "load_expert_memories" which recieves three parameters: 1) "path", string with path to data. 2) "load_action", boolean to load or not the actions. We can performs IRL training the discriminator in differenciate only the states reached by an expert from the states reached by an agent or to differenciante the the state-action pairs from the expert and agent. 3) "n_stack" defines how many temporal steps will be stacked to form the state when using the discriminator. We can used stacked states in the agent and not in the discriminator or we can use it for both.

In [ ]:
exp_path = "tutorials/tf_tutorials/expert_demonstrations/ExpertLunarLander.pkl"

use_expert_actions = True
discriminator_stack = 3
expert_traj = load_expert_memories(exp_path, load_action=use_expert_actions, n_stack=discriminator_stack)

## Build the IRL Problem

As well as a RL problem, an IRL problem have some parameter detailed bellow:

* rl_problem: RL problem defined before. This is formed by an environment an a RL agent.
* expert_traj: RL problem defined before. This is formed by an environment an a RL agent.
* lr_disc: learning rate for training the discriminator neural network.
* batch_size_disc: Size of the batches used for training the discriminator neural network. 
* epochs_disc: Number of epochs fr training the discriminator in each algorithm iteration.
* val_split_disc: Validation split of the data used when training the discriminator.
* agent_collect_iter: Number of episodes of agent's experiences collection in each iteration of the algorithm. Each iteration of the algorithm, the RL agent is run n epochs in exploitation mode to collect data to train the discriminator network.
* agent_train_iter: Number of episodes of RL training procedure in each iteration of the algorithm. Each iteration of the algorithm, the RL agent is trained over the reward infered by the discriminator network. 
* n_stack_disc: number of stacked timesteps to for the state in the discriminator input.
* net_architecture: net architecture defined before.
* use_expert_actions: Flag for use or not actions for training the discriminator. If true, the discriminator will recieve as input state-action pairs. If False, the discriminator will recieve as inputs states.
* tensorboard_dir: path to folder for store tensorboard summaries.

The most noticeable new parameters are the "rl_problem" and "expert_traj". "rl_problem" is the RL problem defined avobe and formed by an agent and an environment and "expert_traj" is the dataset of expert experiences. We provide utilities for saving and load the expert trajectories in "IL_Problem.base.utils.callbacks.py".

In [ ]:
irl_problem = DeepIRL(rl_problem, expert_traj, lr_disc=1e-5, batch_size_disc=128, epochs_disc=2, val_split_disc=0.1,
                      agent_collect_iter=10, agent_train_iter=25, n_stack_disc=discriminator_stack,
                      net_architecture=irl_net_architecture, use_expert_actions=use_expert_actions, tensorboard_dir="logs")

## Solving the IRL Problem

As we always do in these series of tutorial, lest solve the instanciated problem, in this case an IRL Problem. The parameter for this function are:

- iterations: Number of IRL iterations. This are the number of iterations of the entire algorithm, this means: 1) Collect agent experiences, 2) train the discriminator and 3) train the RL agent.
- render: If render or not the environment during the process.
- max_step_epi: Limits the number of steps  of each episode.
- render_after: render after n iterations.
- skip_state: State skipping technique by Mnih, V., Kavukcuoglu, K., Silver, D. et al. Human-level control through deep reinforcement learning. Nature 518, 529–533 (2015).
            https://doi.org/10.1038/nature14236If.

In [ ]:
print("Entrenamiento de agente con aprendizaje por imitación")
irl_problem.solve(10, render=False, max_step_epi=None, render_after=9, skip_states=1)

In [ ]:
rl_problem.test(10)

In [ ]:
agent_saver.save(agent, 'agent_ppo.json')

# Takeaways

- We saw a resume of the three imitation learning algorithms included in the library.
- We learned how to load expert demonstrations.
- We learned how to use DeepIRL algorithm.
- We saw how DeepIRL is capable of training every RL agents.
- We trained a PPO agent through DeepIRL.